In [25]:
import tensorflow as tf
import numpy as np
import json
import os
import csv
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
ML_Summer_School_ID = os.getenv('ML_Summer_School_ID')
print("Your Sudent ID is: " + ML_Summer_School_ID)


Your Sudent ID is: ML051_Kaung_Khant_Paing


In [27]:
BATCH_SIZE  = 32
input_img_size = (128,128)
input_shape = (128,128,3)


test_dir = r'/mnt/d/Learning/WAI/ML_Summer/Practical-ML-by-WAI/6_deep_learning/CNN/cats_and_dogs_filtered/test'
model_path = f"./binary_classification/{ML_Summer_School_ID}_model.h5"
class_index_path = f'./binary_classification/{ML_Summer_School_ID}_class_indices.json'
mark_path = "./binary_classification/marks.txt"


In [28]:
model = tf.keras.models.load_model(model_path)
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 127008)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │    32,514,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,515,459 (124.04 MB)

 Trainable params: 32,515,457 (124.04 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

None


In [29]:
# Load class indices from the JSON file
with open(class_index_path, 'r') as f:
    class_indices = json.load(f)
print("Class indices loaded:", class_indices)


Class indices loaded: {'cats': 0, 'dogs': 1}


In [30]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=input_img_size,
    batch_size=BATCH_SIZE,
    shuffle=False,  # Important: preserve label order for evaluation
    label_mode='int'  # returns integer labels
)


Found 248 files belonging to 2 classes.


In [31]:
# === Predict ===
y_true = []
y_pred = []


for images, labels in test_ds:
    predictions = model.predict(images)

    # If binary classification
    if predictions.shape[1] == 1:
        preds = (predictions > 0.5).astype(int).flatten()
    else:  # multi-class
        preds = np.argmax(predictions, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds)

# === Evaluate Accuracy ===
y_true = np.array(y_true)
y_pred = np.array(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step


In [32]:
accuracy = np.mean(y_true == y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 45.97%


In [ ]:
# with open(mark_path, "w") as f:
#     f.write(str(int(accuracy * 100)))